In [107]:
import cv2
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from skimage import feature
from sklearn.metrics import (f1_score, precision_score, recall_score, 
accuracy_score, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import (RBF, Matern)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier

In [108]:
def criarListaImagens(path, img_size):
    lista = []
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (img_size, img_size))
            lista.append([new_array, int(img[2:5])%2])
        except Exception as e:
            print(e)
            pass
    return lista

#----------------------------------------------------------------------#

def criaListaTreinoTeste(lista):
  classes = []
  imagens = []
  for imagem in lista:
    imagens.append(imagem[0])
    classes.append(imagem[1])
  return imagens, classes

#----------------------------------------------------------------------#

def obtemCanaisCoresRGB(listaOrigem):
  listaDestino = []
  for imagem in listaOrigem:
    aux = cv2.split(imagem)
    aux.append(cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY))
    listaDestino.append(aux)

  return listaDestino

#----------------------------------------------------------------------#

def obterCanaisCores(listaOrigem, cvt):
  listaDestino = []
  for imagem in listaOrigem:    
    i = cv2.cvtColor(imagem, cvt)
    aux = cv2.split(i)
    aux = cv2.split(i)
    aux.append(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY))
    listaDestino.append(aux)

  return listaDestino

#----------------------------------------------------------------------#

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

#----------------------------------------------------------------------#

def obtemHistogramaCanais(listaImagens, lbp):
  canal_1 = []
  canal_2 = []
  canal_3 = []
  canal_cinza = []
  for imagem in listaImagens:
    canal_1.append(lbp.describe(imagem[0]))
    canal_2.append(lbp.describe(imagem[1]))
    canal_3.append(lbp.describe(imagem[2]))
    canal_cinza.append(lbp.describe(imagem[3]))
  return canal_1, canal_2, canal_3, canal_cinza

#----------------------------------------------------------------------#

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
        max_iter=-1, probability=False, random_state=109, shrinking=True, tol=0.001,
        verbose=False),
    SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf', max_iter=-1,
        probability=False, random_state=109, shrinking=True, tol=0.001,verbose=False),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(class_weight=None, criterion='entropy',
        max_depth=3, max_features=None, max_leaf_nodes=None,
        min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=50, min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        random_state=None, splitter='best'),
    RandomForestClassifier (bootstrap=False, class_weight=None,
        criterion='gini', max_depth=50, max_features='sqrt',
        max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=1, min_samples_split=2,
        min_weight_fraction_leaf=0.0, n_estimators=550,
        n_jobs=None, oob_score=False, random_state=None,
        verbose=0, warm_start=False),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

#----------------------------------------------------------------------#

def xgboost(listaTreino, listaTeste, classesTreino, classesTeste):
    m = XGBClassifier(colsample_bytree= 0.6, 
        gamma= 5, learning_rate= 0.15, max_depth= 5, 
        min_child_weight= 1, n_estimators= 100, subsample=0.8, eta= 0.3 ).fit(listaTreino, classesTreino)
    preds = m.predict(listaTeste)
    y_pred = m.predict(listaTeste)     
    acc = accuracy_score(classesTeste, y_pred)
    f1  = f1_score(classesTeste, y_pred, average='weighted')
    cm  = confusion_matrix(classesTeste, y_pred)
    ps  = precision_score(classesTeste, y_pred, average='weighted')
    rs  = recall_score(classesTeste, y_pred, average='weighted')
    return acc, f1, cm, ps, rs

#----------------------------------------------------------------------#

def utilizarClassificadores(listaTreino, classesTreino, listaTeste, classesTeste, namesClassifiers, classifiers):
    listaAccuracy = {}
    listaCM = {}
    listaF1Score = {}
    listaPS = {}
    listaRS = {}
    for i, c in enumerate(classifiers):
        m = c.fit(listaTreino, classesTreino)
        y_pred = m.predict(listaTeste)         
        listaAccuracy[namesClassifiers[i]] = accuracy_score(classesTeste, y_pred)
        listaF1Score[namesClassifiers[i]]  = f1_score(classesTeste, y_pred, average='weighted')
        listaCM[namesClassifiers[i]]       = confusion_matrix(classesTeste, y_pred)
        listaPS[namesClassifiers[i]]       = precision_score(classesTeste, y_pred, average='weighted')
        listaRS[namesClassifiers[i]]       = recall_score(classesTeste, y_pred, average='weighted')
    listaAccuracy["XGBoost"], listaF1Score["XGBoost"], listaCM["XGBoost"], listaPS["XGBoost"], listaRS["XGBoost"] = xgboost(np.asarray(listaTreino), np.asarray(listaTeste), np.asarray(classesTreino), np.asarray(classesTeste))
    return listaAccuracy , listaF1Score, listaCM, listaPS, listaRS



In [111]:
data_dir = r"C:\Users\Rosana\Documents\Mestrado\DataSets\RIM_ONE_v2_\imagens"
img_size = 200
listaImagens = criarListaImagens(data_dir, img_size)

In [112]:
imagens, classes = criaListaTreinoTeste(listaImagens)
#imagens_treino, imagens_teste, classes_treino, classes_teste = train_test_split(imagens, classes, test_size=0.33, random_state=42)

In [102]:
normal = 0
glaucoma = 0
for i in classes_treino:
    if (i == 0):
        glaucoma += 1
    else:
        normal += 1
        
print(normal)
print(glaucoma)

177
127


In [103]:
listaRgbTreino = obtemCanaisCoresRGB(imagens_treino)
listaHsvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2HSV)
listaLabTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LAB)
listaLuvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LUV)

listaRgbTeste = obtemCanaisCoresRGB(imagens_teste)
listaHsvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2HSV)
listaLabTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LAB)
listaLuvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LUV)

In [104]:
def classificarImagens(pontos, raio):
    _lbp = LocalBinaryPatterns(pontos, raio)

    lista_treino_rgb_b, lista_treino_rgb_g, lista_treino_rgb_r, lista_treino_rgb_gray = obtemHistogramaCanais(listaRgbTreino, _lbp)
    lista_treino_hsv_h, lista_treino_hsv_s, lista_treino_hsv_v, _ = obtemHistogramaCanais(listaHsvTreino, _lbp)
    lista_treino_lab_l, lista_treino_lab_a, lista_treino_lab_b, _ = obtemHistogramaCanais(listaLabTreino, _lbp)
    lista_treino_luv_l, lista_treino_luv_u, lista_treino_luv_v, _ = obtemHistogramaCanais(listaLuvTreino, _lbp)

    lista_teste_rgb_b, lista_teste_rgb_g, lista_teste_rgb_r, lista_teste_rgb_gray = obtemHistogramaCanais(listaRgbTeste, _lbp)
    lista_teste_hsv_h, lista_teste_hsv_s, lista_teste_hsv_v, _ = obtemHistogramaCanais(listaHsvTeste, _lbp)
    lista_teste_lab_l, lista_teste_lab_a, lista_teste_lab_b, _ = obtemHistogramaCanais(listaLabTeste, _lbp)
    lista_teste_luv_l, lista_teste_luv_u, lista_teste_luv_v, _ = obtemHistogramaCanais(listaLuvTeste, _lbp)

    linha = []
    listaF1Score = []
    listaCM = [] 
    conf_matrix = []
    listaPS = []
    listaRS = []

    #('------------------ RGB ----------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_gray, classes_treino, lista_teste_rgb_gray, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - R ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_r, classes_treino, lista_teste_rgb_r, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - G ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_g, classes_treino, lista_teste_rgb_g, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - B ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_b, classes_treino, lista_teste_rgb_b, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)

    #('------------------ HSV - H ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_h, classes_treino, lista_teste_hsv_h, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - S ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_s, classes_treino, lista_teste_hsv_s, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - V ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_v, classes_treino, lista_teste_hsv_v, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)


    #('------------------ LAB - L ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_l, classes_treino, lista_teste_lab_l, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ LAB - A ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_a, classes_treino, lista_teste_lab_a, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ LAB - B ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_lab_b, classes_treino, lista_teste_lab_b, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    
    #('------------------ LUV - L ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_l, classes_treino, lista_teste_luv_l, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
    
    #('------------------ LUV - U ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_u, classes_treino, lista_teste_luv_u, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
    
    #('------------------ LUV - V ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_luv_v, classes_treino, lista_teste_luv_v, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)    
    
    return linha, listaF1Score, listaCM, listaPS, listaRS

In [105]:
def obterResultados (pontos, raio):
    linhas, listaF1Score, listaCM, listaPS, listaRS = classificarImagens(pontos, raio)
    print('------ %s pontos - %s raio ------' % (pontos, raio))
    print('--- Acuracia ---')
    display(pd.DataFrame(linhas))
    print('--- F1 ---')
    display(pd.DataFrame(listaF1Score))
    print('--- Precision Score ---')
    display(pd.DataFrame(listaPS))
    print('--- Recall Score ---')
    display(pd.DataFrame(listaRS))
    print('--- Matriz confusao ---')
    display(pd.DataFrame(listaCM))
    print('---------------------------------')
    

In [106]:
obterResultados(4,1)
obterResultados(8,1)
obterResultados(8,2)
obterResultados(12,2)
obterResultados(16,2)
obterResultados(16,3)
obterResultados(24,3)
obterResultados(24,8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined 

------ 4 pontos - 1 raio ------
--- Acuracia ---


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,0.728477,0.701987,0.761589,0.807947,0.741722,0.768212,0.516556,0.741722,0.728477,0.708609,0.807947
1,0.768212,0.675497,0.748344,0.774834,0.761589,0.774834,0.516556,0.748344,0.695364,0.708609,0.781457
2,0.768212,0.728477,0.827815,0.814570,0.728477,0.754967,0.516556,0.781457,0.715232,0.649007,0.807947
3,0.748344,0.728477,0.794702,0.794702,0.735099,0.741722,0.516556,0.701987,0.682119,0.715232,0.788079
4,0.675497,0.642384,0.642384,0.635762,0.615894,0.642384,0.516556,0.562914,0.622517,0.516556,0.668874
5,0.741722,0.701987,0.801325,0.741722,0.814570,0.735099,0.516556,0.754967,0.675497,0.748344,0.794702
6,0.768212,0.675497,0.748344,0.774834,0.761589,0.781457,0.516556,0.748344,0.695364,0.721854,0.781457
7,0.754967,0.655629,0.748344,0.788079,0.728477,0.768212,0.516556,0.761589,0.708609,0.774834,0.788079
8,0.761589,0.688742,0.715232,0.715232,0.715232,0.741722,0.523179,0.695364,0.715232,0.615894,0.721854
9,0.622517,0.748344,0.774834,0.768212,0.721854,0.721854,0.516556,0.688742,0.741722,0.629139,0.748344


--- F1 ---


...

--- Recall Score ---


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,0.721854,0.754967,0.801325,0.807947,0.735099,0.794702,0.523179,0.774834,0.688742,0.781457,0.821192
1,0.708609,0.794702,0.827815,0.821192,0.721854,0.821192,0.516556,0.801325,0.741722,0.774834,0.814570
2,0.708609,0.788079,0.788079,0.788079,0.735099,0.788079,0.516556,0.788079,0.695364,0.735099,0.788079
3,0.622517,0.675497,0.741722,0.761589,0.576159,0.708609,0.516556,0.721854,0.589404,0.715232,0.701987
4,0.668874,0.721854,0.688742,0.728477,0.668874,0.768212,0.516556,0.629139,0.695364,0.741722,0.781457
5,0.589404,0.615894,0.695364,0.715232,0.596026,0.675497,0.516556,0.655629,0.629139,0.649007,0.688742
6,0.708609,0.794702,0.827815,0.821192,0.721854,0.827815,0.516556,0.801325,0.741722,0.781457,0.814570
7,0.715232,0.794702,0.774834,0.801325,0.682119,0.794702,0.516556,0.748344,0.728477,0.774834,0.774834
8,0.715232,0.847682,0.860927,0.867550,0.768212,0.854305,0.516556,0.754967,0.754967,0.728477,0.827815
9,0.761589,0.768212,0.768212,0.801325,0.708609,0.788079,0.516556,0.715232,0.655629,0.761589,0.754967


--- Matriz confusao ---


,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,XGBoost
0,"[[45, 28], [14, 64]]","[[45, 28], [9, 69]]","[[52, 21], [9, 69]]","[[50, 23], [6, 72]]","[[56, 17], [23, 55]]","[[52, 21], [10, 68]]","[[1, 72], [0, 78]]","[[49, 24], [10, 68]]","[[36, 37], [10, 68]]","[[52, 21], [12, 66]]","[[54, 19], [8, 70]]"
1,"[[40, 33], [11, 67]]","[[48, 25], [6, 72]]","[[53, 20], [6, 72]]","[[52, 21], [6, 72]]","[[38, 35], [7, 71]]","[[52, 21], [6, 72]]","[[0, 73], [0, 78]]","[[53, 20], [10, 68]]","[[45, 28], [11, 67]]","[[47, 26], [8, 70]]","[[49, 24], [4, 74]]"
2,"[[40, 33], [11, 67]]","[[50, 23], [9, 69]]","[[49, 24], [8, 70]]","[[49, 24], [8, 70]]","[[52, 21], [19, 59]]","[[52, 21], [11, 67]]","[[0, 73], [0, 78]]","[[52, 21], [11, 67]]","[[37, 36], [10, 68]]","[[42, 31], [9, 69]]","[[50, 23], [9, 69]]"
3,"[[30, 43], [14, 64]]","[[33, 40], [9, 69]]","[[43, 30], [9, 69]]","[[47, 26], [10, 68]]","[[18, 55], [9, 69]]","[[39, 34], [10, 68]]","[[0, 73], [0, 78]]","[[41, 32], [10, 68]]","[[25, 48], [14, 64]]","[[35, 38], [5, 73]]","[[42, 31], [14, 64]]"
4,"[[37, 36], [14, 64]]","[[41, 32], [10, 68]]","[[38, 35], [12, 66]]","[[41, 32], [9, 69]]","[[34, 39], [11, 67]]","[[50, 23], [12, 66]]","[[0, 73], [0, 78]]","[[40, 33], [23, 55]]","[[41, 32], [14, 64]]","[[48, 25], [14, 64]]","[[48, 25], [8, 70]]"
5,"[[31, 42], [20, 58]]","[[24, 49], [9, 69]]","[[37, 36], [10, 68]]","[[38, 35], [8, 70]]","[[31, 42], [19, 59]]","[[41, 32], [17, 61]]","[[0, 73], [0, 78]]","[[36, 37], [15, 63]]","[[29, 44], [12, 66]]","[[37, 36], [17, 61]]","[[41, 32], [15, 63]]"
6,"[[40, 33], [11, 67]]","[[48, 25], [6, 72]]","[[53, 20], [6, 72]]","[[52, 21], [6, 72]]","[[38, 35], [7, 71]]","[[53, 20], [6, 72]]","[[0, 73], [0, 78]]","[[53, 20], [10, 68]]","[[45, 28], [11, 67]]","[[48, 25], [8, 70]]","[[49, 24], [4, 74]]"
7,"[[39, 34], [9, 69]]","[[50, 23], [8, 70]]","[[47, 26], [8, 70]]","[[52, 21], [9, 69]]","[[46, 27], [21, 57]]","[[51, 22], [9, 69]]","[[0, 73], [0, 78]]","[[49, 24], [14, 64]]","[[45, 28], [13, 65]]","[[49, 24], [10, 68]]","[[48, 25], [9, 69]]"
8,"[[43, 30], [13, 65]]","[[56, 17], [6, 72]]","[[57, 16], [5, 73]]","[[59, 14], [6, 72]]","[[64, 9], [26, 52]]","[[60, 13], [9, 69]]","[[0, 73], [0, 78]]","[[49, 24], [13, 65]]","[[47, 26], [11, 67]]","[[41, 32], [9, 69]]","[[54, 19], [7, 71]]"
9,"[[49, 24], [12, 66]]","[[49, 24], [11, 67]]","[[50, 23], [12, 66]]","[[53, 20], [10, 68]]","[[62, 11], [33, 45]]","[[53, 20], [12, 66]]","[[0, 73], [0, 78]]","[[47, 26], [17, 61]]","[[38, 35], [17, 61]]","[[52, 21], [15, 63]]","[[51, 22], [15, 63]]"


---------------------------------
